## K-均值聚类支持函数

In [1]:
from numpy import *

def loadDataSet(fileName):
    dataMat = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float,curLine))
        dataMat.append(fltLine)
    return dataMat
def disEclud(vecA,vecB):
    import math
    return math.sqrt(sum(power(vecA - vecB,2)))
def randCent(dataSet,k):
    n = shape(dataSet)[1]
    centroids = mat(zeros((k,n)))
    for j in range(n):
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k,1)
    return centroids

In [2]:
datMat = mat(loadDataSet('testSet.txt'))

In [3]:
min(datMat[:,0])

matrix([[-5.379713]])

## k-均值聚类算法

In [4]:
# K-均值聚类算法
def Kmeans(dataSet,k,distMeans = disEclud,createCent = randCent):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroids = createCent(dataSet,k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):
            minDist = inf
            minIndex = -1
            for j in range(k):
                distJI = distMeans(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI
                    minIndex = j
            if clusterAssment[i,0] != minIndex:
                clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        print(centroids)
        for cent in range(k):
            ptsInClust = dataSet[nonzero(clusterAssment[:,0].A == cent)[0]]
            centroids[cent,:] = mean(ptsInClust,axis = 0)
    return centroids,clusterAssment

In [5]:
datMat = mat(loadDataSet('testSet.txt'))

In [6]:
myCentroide,clustAssing = Kmeans(datMat,4)

[[ 1.43349662  0.76156471]
 [ 3.29914386  4.0639438 ]
 [ 0.54510913  1.2763215 ]
 [ 2.59808578 -2.0546357 ]]
[[ 2.5788746   1.1559532 ]
 [ 2.624524    3.53268467]
 [-2.92009409  0.60553546]
 [ 1.66608012 -3.02478724]]
[[ 3.15787957  0.54371129]
 [ 2.32648653  3.48573529]
 [-3.06957486  0.04470084]
 [ 2.29630447 -3.17754284]]
[[ 3.26860533  0.30704483]
 [ 2.12141968  3.44538163]
 [-3.079971   -0.18431611]
 [ 2.54173689 -3.11892933]]
[[ 3.26860533  0.30704483]
 [ 1.80435343  3.39147748]
 [-3.18695357 -0.35938491]
 [ 2.54173689 -3.11892933]]
[[ 3.26860533  0.30704483]
 [ 1.6457145   3.370942  ]
 [-3.231108   -0.45641674]
 [ 2.54173689 -3.11892933]]


In [7]:
myCentroide

matrix([[ 3.26860533,  0.30704483],
        [ 1.6457145 ,  3.370942  ],
        [-3.231108  , -0.45641674],
        [ 2.54173689, -3.11892933]])

## 二分K-均值聚类算法

In [11]:

# 二分K-均值聚类算法
def biKmeans(dataSet,k,distMeas = disEclud):
    m = shape(dataSet)[0]
    clusterAssment = mat(zeros((m,2)))
    centroid0 = mean(dataSet,axis = 0).tolist()[0]
    centList = [centroid0]
    for j in range(m):
        clusterAssment[j,1] = distMeas(mat(centroid0),dataSet[j,:])**2
    while (len(centList) < k):
        lowestSSE = inf
        for i in range(len(centList)):
            ptsInCurrCluster = dataSet[nonzero(clusterAssment[:,0].A == i)[0],:]
            centroidMat,splitClustAss = Kmeans(ptsInCurrCluster,2,distMeas)
            sseSplit = sum(splitClustAss[:,1])
            sseNotSplit = sum(clusterAssment[nonzero(clusterAssment[:,0].A != i)[0],1])
            print("sseSplit, and not Split: ",sseSplit,sseNotSplit)
            if (sseSplit + sseNotSplit) < lowestSSE:
                bestCentToSplit = i
                bestNewCents = centroidMat
                bestClustAss = splitClustAss.copy()
                lowestSSE = sseSplit + sseNotSplit
        bestClustAss[nonzero(bestClustAss[:,0].A == 1)[0],0] = len(centList)
        bestClustAss[nonzero(bestClustAss[:,0].A == 0)[0],0] = bestCentToSplit
        print("the bestCentToSplit is: ",bestCentToSplit)
        print("the len of bestClustAss is: ",len(bestClustAss))
        centList[bestCentToSplit] = bestNewCents[0,:].tolist()[0]
        centList.append(bestNewCents[1,:].tolist()[0])
        clusterAssment[nonzero(clusterAssment[:,0].A == bestCentToSplit)[0],:] = bestClustAss
    return mat(centList),clusterAssment

In [12]:
datMat3 = mat(loadDataSet("testSet2.txt"))

In [13]:
centList,myNewAssments = biKmeans(datMat3,3)

[[-1.76041866  1.17238886]
 [-0.47142374  4.59350951]]
[[-1.22761874 -0.91968645]
 [ 0.98595917  3.51827886]]
[[-0.97166321 -1.96674883]
 [ 0.38490422  3.35338275]]
[[-0.62407076 -2.58281924]
 [ 0.09338772  3.27587208]]
[[-0.45965615 -2.7782156 ]
 [-0.00675605  3.22710297]]
sseSplit, and not Split:  453.0334895807502 0.0
the bestCentToSplit is:  0
the len of bestClustAss is:  60
[[-1.70785245 -3.91594994]
 [-1.75471992 -2.98411797]]
[[ 0.34949733 -3.6403825 ]
 [-0.80643621 -2.4087155 ]]
[[ 0.34658275 -3.48379675]
 [-0.99714875 -2.30782817]]
[[ 0.35496167 -3.36033556]
 [-1.12616164 -2.30193564]]
sseSplit, and not Split:  12.753263136887313 423.8762401366249
[[ 0.96972241  0.49272599]
 [-4.64003538  4.01079115]]
[[ 2.76275171  3.12704005]
 [-3.06779095  3.33769884]]
[[ 2.93386365  3.12782785]
 [-2.94737575  3.3263781 ]]
sseSplit, and not Split:  77.59224931775066 29.15724944412535
the bestCentToSplit is:  1
the len of bestClustAss is:  40


In [14]:
# 质心
centList

matrix([[-0.45965615, -2.7782156 ],
        [ 2.93386365,  3.12782785],
        [-2.94737575,  3.3263781 ]])

## 示例：对地图上的点进行聚类